In [1]:
%load_ext autoreload
%aimport -pandas, -numpy
%autoreload 2
import pandas as pd
import numpy as np
import arrow
from dfsm import msgFSM

In [2]:
import dmyplant2
dmyplant2.cred()
mp = dmyplant2.MyPlant(0)


In [3]:
dval = dmyplant2.Validation.load_def_csv("input2.csv")
i = 5
vale = dval.iloc[i]
#[['serialNumber','Validation Engine','val start']]
vale

n                                                    5
Validation Engine                           REGENSBURG
serialNumber                                   1243362
val start                          2020-09-07 00:00:00
oph@start                                        62765
starts@start                                       NaN
Asset ID                                      157701.0
Old PU first replaced OPH                       4913.0
Old PUs replaced before upgrade                    1.0
Name: 5, dtype: object

In [4]:
sn = 1243362
e=dmyplant2.Engine.from_sn(mp,sn, valstart='2020-09-07')
pd.DataFrame.from_dict(e.dash, orient='index').T

,Name,Engine ID,Design Number,Engine Type,Engine Version,P,P_nom,BMEP,serialNumber,id,Count_OpHour,val start,oph@start,oph parts,LOC
0,BMW Regensburg,M2,I448,616,J01,16,2744.9,22.0,1243362,157701,71738.0,2020-09-07,0,71738,None


In [5]:
res = e.get_keyItem_data('dataItems','Count_OpHour')
{ res['id'] : [res['name'],res['unit']] }

{161: ['Count_OpHour', 'h']}

In [6]:
e.get_keyItem('Count_OpHour')

{'id': 161,
 'name': 'Count_OpHour',
 'timestamp': 1645360471187,
 'unit': 'h',
 'value': 71805.0}

In [7]:
e.get_keyId('Count_OpHour')

161

In [8]:
datal = ['Count_OpHour','Power_PowerAct','Hyd_PressCrankCase','Hyd_PressOilDif']
dat = e.get_dataItems(['Count_OpHour','Power_PowerAct','Hyd_PressCrankCase','Hyd_PressOilDif'])

In [9]:
#def _batch_hist_dataItems(self, itemIds={161: ['CountOph', 'h']}, p_limit=None, p_from=None, p_to=None, timeCycle=3600,
#                              assetType='J-Engine', includeMinMax='false', forceDownSampling='false'):
e._batch_hist_dataItems(dat, p_from=1645281269000, p_to=1645281269000).iloc[0]

time                        1645281269000
Hyd_PressCrankCase             -15.344637
Power_PowerAct                1976.519094
Hyd_PressOilDif                  0.572489
Count_OpHour                 71782.425063
datetime              2022-02-19 14:34:29
Name: 0, dtype: object

In [10]:
def fetch_dataItems(e, items, ts):
    itemIds = e.get_dataItems(items)
    tdj = ','.join([str(s) for s in itemIds])
    url=fr"/asset/{e['id']}/history/batchdata?assetType=J-Engine&from={ts}&to={ts}&dataItemIds={tdj}&timeCycle=30"
    data =  e._mp.fetchdata(url)
    # # restructure data to dict
    # ds = {}
    # ds['labels'] = ['timestamp'] + [itemIds[x][0] for x in data['columns'][1]]
    # ds['data'] = [[r[0]] + [rr[0] for rr in r[1]] for r in [data['data'][0]]]
    # ds['index'] = [r[0] for r in data['data'][0] ]
    # # import to Pandas DataFrame
    # df = pd.DataFrame(ds['data'], columns=ds['labels'])
    # return df
    return data

mdata = fetch_dataItems(e, ['Count_OpHour','Power_PowerAct','Hyd_PressCrankCase','Hyd_PressOilDif'], 1645281269000)

In [11]:
itemIds = e.get_dataItems(['Count_OpHour','Power_PowerAct','Hyd_PressCrankCase','Hyd_PressOilDif'])
data = {
    'columns': ['time', [161, 217, 16546, 102]],
    'data': 
    [
        [
            1645281269000,
            [
                [71782.0],
                [-15.90],
                [0.5725],
                [2009.0]
            ]
        ],
        [
            1645281269001,
            [
                [71783.0],
                [-15.91],
                [0.5726],
                [2009.5]
            ]
        ]
    ]
}
data

{'columns': ['time', [161, 217, 16546, 102]],
 'data': [[1645281269000, [[71782.0], [-15.9], [0.5725], [2009.0]]],
  [1645281269001, [[71783.0], [-15.91], [0.5726], [2009.5]]]]}

In [12]:
d = {}
ldata = [data['data'][0]]
for i, key in enumerate(data['columns']):
    if isinstance(key, str):
        d[key] = [value[i] for value in ldata]
    if isinstance(key, list):
        for j, skey in enumerate(key):
            d[itemIds[skey][0]] = [ value[i][j][0] for value in ldata ]
from pprint import pprint as pp
pd.DataFrame(d)

,time,Count_OpHour,Hyd_PressCrankCase,Hyd_PressOilDif,Power_PowerAct
0,1645281269000,71782.0,-15.9,0.5725,2009.0


In [13]:
d = []
from pprint import pprint as pp
for i, key in enumerate(data['columns']):
    d.append({key: [value[i] for value in data['data']]} if isinstance(key, str) else \
         {itemIds[skey][0]:[value[i][j][0] for value in data['data']] for j,skey in enumerate(key)})
{**d[0] ,**d[1]}

{'time': [1645281269000, 1645281269001],
 'Count_OpHour': [71782.0, 71783.0],
 'Hyd_PressCrankCase': [-15.9, -15.91],
 'Hyd_PressOilDif': [0.5725, 0.5726],
 'Power_PowerAct': [2009.0, 2009.5]}

In [14]:
d =  [{key: [value[i] for value in [data['data'][0]]]} if isinstance(key, str) \
    else {itemIds[skey][0]:[value[i][j][0] for value in [data['data'][0]]] \
        for j,skey in enumerate(key)} \
            for i, key in enumerate(data['columns'])]
pd.DataFrame([d[0]|dd for dd in d[1:]][0])

,time,Count_OpHour,Hyd_PressCrankCase,Hyd_PressOilDif,Power_PowerAct
0,1645281269000,71782.0,-15.9,0.5725,2009.0


In [20]:
dtime = {'timestamp' : [value[0] for value in [data['data'][0]]]}
ddata = {itemIds[skey][0]:[value[1][j][0] for value in [data['data'][0]]] for j,skey in enumerate(data['columns'][1])}
d = dtime|ddata
d

{'timestamp': [1645281269000],
 'Count_OpHour': [71782.0],
 'Hyd_PressCrankCase': [-15.9],
 'Hyd_PressOilDif': [0.5725],
 'Power_PowerAct': [2009.0]}

In [23]:
e.fetch_dataItems(1645281269000,['Count_OpHour','Various_Values_SpeedAct','Power_PowerAct','Power_SetPower','Hyd_PressCrankCase','Hyd_PressCoolWat','rP_Ramp_Set'])

,timestamp,Various_Values_SpeedAct,rP_Ramp_Set,Count_OpHour,Hyd_PressCoolWat,Hyd_PressCrankCase,Power_SetPower,Power_PowerAct
0,1645281269000,1499.0,0.63,71782.007533,1.56,-15.90832,2006.641167,2009.9376


In [ ]:
e.asset['validation']

In [ ]:
e['Power_PowerNominal']

In [ ]:
#mp._fetch_installed_base()
e.lookup_Installed_Fleet(mp,e._sn)

In [ ]:

e.get_messages()

In [ ]:
for k in sorted(e.properties.keys()):
    print(k)

In [ ]:
100 / e['rP_Ramp_Set']

In [ ]:
'Ramp' in 'rP_Ramp_Act'

In [ ]:
for i, k in enumerate(sorted(e.properties.keys())):
    print(f"{i:03} {k}: {e[k]}")

In [ ]:
for i, k in enumerate(sorted(e.dataItems.keys())):
    if not k.startswith('Para_') and e[k] != None:
        #if k.startswith('Knock_KLS98_Knock'):
            print(f"{i:04} {k}: {e[k]}")

In [ ]:
for k in sorted(e.dataItems.keys()):
    if 'Ramp' in k:
        print(k, e[k])

In [ ]:
for i, k in enumerate(sorted(e.dataItems.keys())):
    if not k.startswith('Para_') and e[k] != None:
        if k[-2:] in [f"{i:02}" for i in range(1,25)]:
            print(f"{i:04} {k}: {e[k]}")